In [2]:
import numpy as np
import pandas as pd
from shapely import geometry
from geopy.distance import geodesic
import shapely.speedups
from tqdm import tqdm
from quetzal.model import stepmodel

In C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none func

# Preparation of the transport network.
## Saves aggregated bus and short-distance rail network.
## Needs PT networks with access and egress.

In [3]:
input_path = '../input_static/'
output_path = '../output/'
model_path = '../model/'

In [3]:
# Loading StepModel with PT networks...
sm = stepmodel.read_json(input_path + 'de_pt_network')
bus = stepmodel.read_json(input_path + 'de_pt_network_bus')
# Loading access and egress
ae = stepmodel.read_json(model_path + 'de_pt_access_egress')

In [4]:
sm.nodes.sample()

,route_type,stop_name,geometry
index,,,
rail_short_n_14385,rail_short_distance,Haste,POINT (9.38910 52.37966)


## Check and correct network integrity

In [7]:
# FIRST: Set time of free-rider links
# Default velocity of 14 m/s for rail links
sm.links.loc[sm.links['time']==0, 'time'] = sm.links.loc[
    sm.links['time']==0, 'geometry'].apply(
        lambda l: int(geodesic(l.coords[0], l.coords[-1]).m)) / 14
# 8 m/s for bus links
bus.links.loc[bus.links['time']==0, 'time'] = bus.links.loc[
    bus.links['time']==0, 'geometry'].apply(
        lambda l: int(geodesic(l.coords[0], l.coords[-1]).m)) / 8

In [8]:
# Check nodeset integrity for later steps to work
try:
    sm.integrity_test_nodeset_consistency()
except AssertionError:
    print('Found {} orphan nodes'.format(len(sm.orphan_nodes)))
    sm.nodes.drop(sm.orphan_nodes, inplace=True)
    # Test integrity again
    sm.integrity_test_nodeset_consistency()

Found 5 orphan nodes
no road_links or road_nodes


In [9]:
# Test sequences
# Use an own function because quetzal's takes ages
def test_sequences(trip):
    assert len(trip)==trip['link_sequence'].max(), \
        'broken sequence in trip {}'.format(trip['trip_id'].unique()[0])

In [10]:
# Fix sequences
# Use an own function because quetzal's takes ages
def fix_sequences(trip):
    trip = trip.sort_values('link_sequence')
    # Check link succession
    ind = list(trip.index)
    for i in range(len(trip.index) - 1):
        try:
            assert trip.loc[ind[i], 'b'] == trip.loc[ind[i+1], 'a'], \
                'broken trip {}: stop {} has no successor link'.format(
                    trip['trip_id'].unique()[0], trip.loc[ind[i], 'b'])
        except AssertionError:
            trip.loc[ind[i+1]:ind[-1], 'trip_id'] = \
                trip.loc[ind[i+1]:ind[-1], 'trip_id'] + '_' + str(i)
    # Repair sequences
    if len(trip) != trip['link_sequence'].max():
        trip['link_sequence'] = trip.groupby('trip_id')['link_sequence'].apply(
            lambda t: [j for j in range(1, len(t.index)+1)]).sum()
    return trip

In [11]:
# Test and save broken sequences
def test_sequences_save(trip):
    if len(trip)!=trip['link_sequence'].max():
        return list(trip.index)

In [12]:
tqdm.pandas()
try:
    sm.links.groupby('trip_id').progress_apply(test_sequences)
except AssertionError:
    links = sm.links.groupby('trip_id').progress_apply(fix_sequences).reset_index(level=0, drop=True)
    links.groupby('trip_id').progress_apply(test_sequences)
    sm.links = links

100%|██████████████████████████████████████████████████████████████████████████| 21118/21118 [00:07<00:00, 2874.18it/s]


In [13]:
try:
    bus.links.groupby('trip_id').progress_apply(test_sequences)
except AssertionError:
    broken_seqs = bus.links.groupby('trip_id').progress_apply(test_sequences_save)
    links = bus.links.loc[broken_seqs.loc[broken_seqs.notna()].sum()
                         ].groupby('trip_id').progress_apply(fix_sequences)
    links.reset_index(level=0, drop=True, inplace=True)
    links.groupby('trip_id').progress_apply(test_sequences)
    bus.links = bus.links.drop(broken_seqs.loc[broken_seqs.notna()].sum()).append(links)

100%|████████████████████████████████████████████████████████████████████████| 212259/212259 [01:19<00:00, 2670.55it/s]


In [14]:
assert len(bus.nodes['route_type'].unique()) == 1

In [15]:
# Divide nodes
print(sm.nodes.shape)
#disagg_nodes = sm.nodes.loc[sm.nodes['route_type']=='rail_short_distance'].append(bus.nodes)
#sm.nodes = sm.nodes.loc[sm.nodes['route_type']!='rail_short_distance']
disagg_nodes = bus.nodes
print(disagg_nodes.shape)

(15394, 3)
(413787, 3)


In [16]:
# Divide links
print(sm.links.shape)
#disagg_links = sm.links.loc[sm.links['route_type']=='rail_short_distance'].append(bus.links)
#sm.links = sm.links.loc[sm.links['route_type']!='rail_short_distance']
disagg_links = bus.links
print(disagg_links.shape)

(213129, 8)
(3211174, 8)


In [17]:
# Number of trips
len(disagg_links['trip_id'].unique())

212259

## Remove unneccessary stops
Bus and short-distance rail service in the GTFS feeds contain trips with not further connected intermediate stops. Thus, the PT network graph can be reduced without loss of information.

In [18]:
sm.links.sample()

,a,b,link_sequence,route_id,route_type,time,trip_id,geometry
index,,,,,,,,
rail_short_133623,rail_short_n_877,rail_short_n_1187,1.0,rail_short_85,rail_short_distance,186.0,rail_short_35780,"LINESTRING (13.38559 52.65491, 13.34054 52.66460)"


In [19]:
# Count the number of links to/from each node
n_links_dict = disagg_links[['a', 'b']].stack().value_counts().to_dict()
connector_set = ae.zone_to_transit[['a', 'b']].append(ae.footpaths[['a', 'b']])
n_connectors_dict = connector_set.stack().value_counts().to_dict()
disagg_nodes['n_links'] = [n_links_dict[i] for i in list(disagg_nodes.index)]
disagg_nodes['n_connectors'] = disagg_nodes.index.map(n_connectors_dict)
disagg_nodes['n_connectors'].replace(np.nan, 0, inplace=True)

In [20]:
disagg_nodes.loc[disagg_nodes.isna().any(axis=1)]

,stop_name,route_type,geometry,n_links,n_connectors
index,,,,,


In [21]:
print(disagg_nodes.loc[disagg_nodes['route_type']=='bus'].shape)
print(disagg_nodes.loc[disagg_nodes['route_type']!='bus'].shape)

(413787, 5)
(0, 5)


In [22]:
disagg_nodes.sample(2)

,stop_name,route_type,geometry,n_links,n_connectors
index,,,,,
bus_n_371528,Artelshofen Zum Pechwirt,bus,POINT (11.49822 49.57359),16,0.0
bus_n_33604,"Reuden (Kem), Kindergarten",bus,POINT (12.59892 51.77179),18,0.0


In [25]:
# Keep interconnected rail trips but drop all bus trips
# without connection to another mode or centroid
agg_nodes = disagg_nodes.loc[
    ((disagg_nodes['route_type']!='bus') &
     ((disagg_nodes['n_links'] > 2) |
      (disagg_nodes['n_connectors'] > 0))
    ) | (
        (disagg_nodes['route_type']=='bus') &
        (disagg_nodes['n_connectors'] > 2)
    )]

In [26]:
print(agg_nodes.loc[agg_nodes['route_type']=='bus'].shape)
print(agg_nodes.loc[agg_nodes['route_type']!='bus'].shape)

(6440, 5)
(0, 5)


### Aggregate links within the broken trips

In [27]:
# Mark dropped nodes
disagg_links['relevant'] = disagg_links['a'].isin(list(agg_nodes.index)) | \
    disagg_links['b'].isin(list(agg_nodes.index))

In [28]:
disagg_links.loc[disagg_links['relevant']].shape

(122800, 9)

In [29]:
disagg_links.sample()

,route_id,route_type,a,b,time,trip_id,link_sequence,geometry,relevant
index,,,,,,,,,
bus_x_2857,bus_x_10663,bus,bus_x_n_424228,bus_x_n_261072,60.0,bus_x_7080,9,"LINESTRING (7.15679 50.23708, 7.15114 50.23412)",False


In [30]:
# Function for aggregating links
def agg_trips(trip):
    # Drop links with missing nodes
    trip_agg = trip.loc[trip['relevant']].sort_values('link_sequence')
    if len(trip_agg.index) == 0:
        # Trip is fully irrelevant
        return
    
    missing_nodes = list(set(list(trip_agg['a'])+list(trip_agg['b'])) -
                         set(agg_nodes.index))
    if len(missing_nodes)==0 and len(trip.index)==len(trip_agg.index):
        # This trip is not affected
        return trip
    
    # Repair link succession
    ind = list(trip_agg.index)
    for i in range(len(ind) - 1):
        if trip_agg.loc[ind[i], 'b'] in missing_nodes:
            trip_agg.loc[ind[i + 1], 'a'] = trip_agg.loc[ind[i], 'a']
            try:
                trip_agg.loc[ind[i + 1], 'geometry'] = geometry.LineString(
                    [agg_nodes.loc[trip_agg.loc[ind[i + 1], 'a'], 'geometry'],
                     agg_nodes.loc[trip_agg.loc[ind[i + 1], 'b'], 'geometry']])
            except KeyError:
                return
            trip_agg.drop(ind[i], inplace=True)
            i = i + 1
    
    ind = list(trip_agg.index)
    if len(trip_agg.index) > 0 and trip_agg.loc[ind[0], 'a'] in missing_nodes:
        # Drop unused first link
        trip_agg = trip_agg.iloc[1:]
    if len(trip_agg.index) > 0 and trip_agg.loc[ind[-1], 'b'] in missing_nodes:
        # Drop unused last link
        trip_agg = trip_agg.iloc[:-1]
    ind = list(trip_agg.index)
    if len(ind) == 0:
        return
    
    # Aggregate travel time
    for i in range(len(ind) - 1):
        try:
            assert trip_agg.loc[ind[i], 'b'] == trip_agg.loc[ind[i+1], 'a'], \
                'broken sequence in trip {}: stop {} has no successor link'.format(
                    trip_agg['trip_id'].unique()[0], trip_agg.loc[ind[i], 'b'])
        except AssertionError:
            # Drop this trip
            return
        if trip_agg.loc[ind[i + 1], 'link_sequence'] - trip_agg.loc[ind[i], 'link_sequence'] > 1:
            trip_agg.loc[ind[i], 'time'] = trip.loc[ind[i]:ind[i+1], 'time'].sum() - \
                trip_agg.loc[ind[i+1], 'time'] # pandas slicing includes both boundaries
    
    # Reindex the sequence numbers
    trip_agg['link_sequence'] = [i for i in range(1, len(trip_agg.index)+1)]
    
    return trip_agg

In [96]:
# Faster variant for using multiple cores
'''import multiprocessing as mp
with mp.Pool(processes=10) as p:
    agg_links = pd.concat(p.map(
        agg_trips, [g for _, g in disagg_links.groupby('trip_id')]))'''

"import multiprocessing as mp\nwith mp.Pool(processes=10) as p:\n    agg_links = pd.concat(p.map(\n        agg_trips, [g for _, g in disagg_links.groupby('trip_id')]))"

In [31]:
agg_links = disagg_links.groupby('trip_id').progress_apply(
    agg_trips).reset_index(level=0, drop=True)

100%|█████████████████████████████████████████████████████████████████████████| 212259/212259 [08:17<00:00, 426.62it/s]


In [32]:
agg_links.drop('relevant', axis=1, inplace=True)
agg_links.shape

(48243, 8)

In [33]:
agg_links.loc[~(agg_links['a'].isin(list(agg_nodes.index))) |
    ~(agg_links['b'].isin(list(agg_nodes.index)))]

,route_id,route_type,a,b,time,trip_id,link_sequence,geometry
index,,,,,,,,
bus_1415622,bus_16202,bus,bus_n_282557,bus_n_347696,1020.0,bus_302856,1,"LINESTRING (11.90602 54.10878, 11.90068 54.11217)"
bus_2310601,bus_12805,bus,bus_n_117738,bus_n_263121,120.0,bus_712299,1,"LINESTRING (11.75354 51.79189, 11.75072 51.79629)"
bus_2477532,bus_5783,bus,bus_n_222182,bus_n_85142,180.0,bus_805166,1,"LINESTRING (7.38907 53.08167, 7.38792 53.08975)"
bus_2699852,bus_16202,bus,bus_n_282557,bus_n_347696,1020.0,bus_937870,1,"LINESTRING (11.90602 54.10878, 11.90068 54.11217)"


In [34]:
# Drop these erronous trips
ids = agg_links.loc[~(agg_links['a'].isin(list(agg_nodes.index))) |
                    ~(agg_links['b'].isin(list(agg_nodes.index)))].index
agg_links = agg_links.loc[~agg_links['trip_id'].isin(list(agg_links.loc[ids, 'trip_id']))]

### Merge aggregated links and nodes with the model

In [35]:
# Re-add links to model
sm.links = sm.links.append(agg_links)
sm.links.shape

(261366, 8)

In [36]:
# Re-add nodes to the model
sm.nodes = sm.nodes.append(agg_nodes)
sm.nodes.shape

(21834, 5)

In [37]:
try:
    sm.integrity_test_nodeset_consistency()
except AssertionError:
    print('Number of orphan nodes: {}'.format(
        len(sm.orphan_nodes)))
    print('Number of missing nodes: {}'.format(
        len(sm.missing_nodes)))

Number of orphan nodes: 688
Number of missing nodes: 0


In [38]:
# Leave these nodes if they connect by more than one
# footpath or access/egress link
assert len(agg_nodes.loc[sm.orphan_nodes].loc[agg_nodes['n_connectors']>1].index) == len(sm.orphan_nodes)

In [39]:
sm.nodes.drop(['n_links', 'n_connectors'], axis=1, inplace=True)

In [40]:
sm.links.groupby('trip_id').progress_apply(test_sequences)

100%|██████████████████████████████████████████████████████████████████████████| 50396/50396 [00:16<00:00, 3096.87it/s]


""


## Map nodes to zones
Needed for the accessibility calculation

In [42]:
sm.zones.sample()

,CNTR_CODE,NUTS_NAME,LEVL_CODE,NUTS_ID,population,area,urbanisation,lau_id,geometry
index,,,,,,,,,
DE236,DE,Neumarkt i. d. OPf.,3,DE236,133561,1343.94,3.0,9373,"POLYGON ((11.55776 49.41904, 11.56383 49.41863..."


In [44]:
# Nodes must be a GeoDataFrame
if 'FID' not in sm.nodes.columns:
    import geopandas as gpd
    sm.nodes = gpd.GeoDataFrame(sm.nodes, crs=sm.epsg)
    shapely.speedups.enable()
    sm.nodes['FID'] = np.nan
    for _, zone in tqdm(sm.zones.iterrows(), total=sm.zones.shape[0]):
        sm.nodes.loc[sm.nodes['geometry'].within(zone['geometry']), 'FID'] = zone['NUTS_ID']
    # Drop zones outside the model zones
    sm.nodes = sm.nodes[sm.nodes['FID'].notna()]
    sm.nodes.shape

100%|████████████████████████████████████████████████████████████████████████████████| 401/401 [00:51<00:00,  7.80it/s]


(21816, 4)

In [45]:
sm.nodes.sample()

,geometry,route_type,stop_name,FID
index,,,,
bus_n_217829,POINT (13.46576 52.49907),bus,Markgrafendamm,DE300


## Save model


In [49]:
# Add bus service to ancilliary
sm.pt_routes = sm.pt_routes.append(bus.pt_routes.loc[
    bus.pt_routes['route_id'].isin(sm.links['route_id'].unique())])
sm.agencies = sm.agencies.append(bus.agencies.loc[
    bus.agencies['agency_id'].isin(sm.pt_routes['agency_id'].unique())])

In [50]:
# Now, we have bus services in the same tables
sm.pt_route_types.append('bus')

In [51]:
# Reduce file size by shortening node index names
sm.nodes['index'] = [i.replace('rail_short_n', 'r_s_n') for i in sm.nodes.index]
sm.nodes.set_index('index', drop=True, inplace=True)
sm.links['a'] = sm.links['a'].apply(lambda n: n.replace('rail_short_n', 'r_s_n'))
sm.links['b'] = sm.links['b'].apply(lambda n: n.replace('rail_short_n', 'r_s_n'))

In [52]:
sm.nodes['index'] = [i.replace('rail_long_node', 'r_l_n') for i in sm.nodes.index]
sm.nodes.set_index('index', drop=True, inplace=True)
sm.links['a'] = sm.links['a'].apply(lambda n: n.replace('rail_long_n', 'r_l_n'))
sm.links['b'] = sm.links['b'].apply(lambda n: n.replace('rail_long_n', 'r_l_n'))

In [53]:
# Shorten link index names
sm.links['index'] = [i.replace('rail_long', 'r_l').replace('rail_short', 'r_s')
                     for i in sm.links.index]
sm.links.set_index('index', drop=True, inplace=True)

In [54]:
# Shorten route type names
type_dict = {'rail_short_distance': 'rail_short', 'rail_long_distance': 'rail_long'}
sm.links['route_type'] = sm.links['route_type'].replace(type_dict)
sm.nodes['route_type'] = sm.nodes['route_type'].replace(type_dict)
sm.pt_route_types = [t.replace('_distance', '') for t in sm.pt_route_types]

In [55]:
sm.links.loc[sm.links['route_type']=='rail_short'].sample()

,a,b,geometry,link_sequence,route_id,route_type,time,trip_id
index,,,,,,,,
r_s_44630,r_s_n_11240,r_s_n_2498,"LINESTRING (8.52873 50.09856, 8.54237 50.10263)",4.0,rail_short_15,rail_short,120.0,rail_short_6939


In [56]:
# Cast columns to int
cols = ['time', 'link_sequence']
sm.links[cols] = sm.links[cols].astype(int)

In [57]:
# Split links in graph and auxiliary information
# for file sizes being compatible with github's size limit
cols = ['link_sequence', 'route_id', 'time', 'trip_id']
auxiliary = sm.links[cols]
sm.links.drop(cols, axis=1, inplace=True)

In [58]:
sm.links.shape

(261366, 4)

In [59]:
# Saving model...
sm.to_json(model_path + 'de_pt_network_agg',
           only_attributes=['zones', 'links', 'nodes', 'pt_route_types'],
           encoding='utf-8')
sm.to_json(model_path + 'de_pt_network_ancillary',
           only_attributes=['agencies', 'pt_routes'],
           encoding='utf-8')

to_hdf(overwriting): 100%|████████████████████████████████████████████████████████████| 38/38 [00:00<00:00, 486.48it/s]


In [60]:
# Save auxiliary information seperately
auxiliary['index'] = auxiliary.index
auxiliary.reset_index(drop=True, inplace=True)
auxiliary.to_json(model_path + 'de_pt_network_agg/links_quetzaldata.json')

## Merge frequencies

The frequency files are too large to be loaded into a 8GB RAM together with the networks. Thus, they are handled afterwards, seperately

In [4]:
# In case your kernel restarted: Load aggregation results
#sm = stepmodel.read_json(model_path + 'de_pt_network_agg')
# Otherwise
sm.links[cols] = auxiliary.set_index('index')[cols]

In [5]:
sm.links.sample()

,a,b,route_type,geometry,link_sequence,route_id,time,trip_id
index,,,,,,,,
coach_21383,coach_n_FLIXBUS:26,coach_n_FLIXBUS:19,coach,"LINESTRING (8.40045 48.99147, 8.47282 49.47808)",2,coach_FLIXBUS:N77,3600,coach_FLIXBUS:N77:2425:14:40:00Z-1


In [6]:
# Load frequencies
bus_frequencies = pd.read_json(input_path + 'de_pt_network_frequencies/frequencies_bus.json')
pt_frequencies = pd.read_json(input_path + 'de_pt_network_frequencies/frequencies_pt.json')

In [7]:
# Drop duplicate index column
pt_frequencies.drop('index', axis=1, errors='ignore', inplace=True)
bus_frequencies.drop('index', axis=1, errors='ignore', inplace=True)

In [8]:
# Replace rail node names
pt_frequencies['stop_id'] = pt_frequencies['stop_id'].apply(lambda n: n.replace('rail_short_n', 'r_s_n'))
pt_frequencies['stop_id'] = pt_frequencies['stop_id'].apply(lambda n: n.replace('rail_long_n', 'r_l_n'))

In [9]:
# Drop frequencies of unused nodes and links
frequencies = pt_frequencies.loc[
    (pt_frequencies['stop_id'].isin(sm.nodes.index)) &
    (pt_frequencies['route_id'].isin(sm.links['route_id']))]

In [10]:
# Add bus
frequencies = frequencies.append(bus_frequencies.loc[
    (bus_frequencies['stop_id'].isin(sm.nodes.index)) &
    (bus_frequencies['route_id'].isin(sm.links['route_id']))
]).reset_index(drop=True)

In [11]:
frequencies.shape

(48135, 3)

In [12]:
frequencies.loc[frequencies.isna().any(axis=1)]

,hour,route_id,stop_id


In [13]:
# Save
frequencies['index'] = frequencies.index
frequencies.to_json(model_path + 'de_pt_network_ancillary/frequencies.json')